# KMACS 데이터 수집

In [ ]:
import warnings
warnings.filterwarnings(action = "ignore")
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import tqdm
from tqdm import tqdm_notebook

#브라우저 꺼짐 방지 
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
#불필요한 에러 메시지 없애기 
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
#연도별 교통사고 페이지
driver.get("https://tmacs.kotsa.or.kr/web/TG/TG200/TG2100S/Tg2102.jsp?mid=S1177#")
time.sleep(1)

In [ ]:
option = driver.find_element(By.XPATH,
        f"/html/body/div[4]/div[2]/div[2]/div[2]/div[1]/div[1]/div/dl[4]/dd/span[3]/input")

option.click()
time.sleep(1)




jijace_list_raw = driver.find_element(By.ID, "jijace") #지자체
jijace_list = jijace_list_raw.find_elements(By.TAG_NAME, "option") #지자체 옵션
jijace_names = [option.text for option in jijace_list] #구 이름 저장
jijace_names = jijace_names[1:]
search = driver.find_element(By.CSS_SELECTOR, "#content > div.cont_box > div.filterForm > div.btn_wrap.btn-2 > a.btn.btn_primary")
excel_btn = driver.find_element(By.CSS_SELECTOR, "#content > div.cont_box > div:nth-child(2) > div.btn_box.type02 > a.btn.exbtn")
for gu in tqdm_notebook(jijace_names):
    jijace_list_raw.send_keys(gu)
    time.sleep(3)
    search.click()
    time.sleep(5)
    excel_btn.click()
    time.sleep(3)


time.sleep(1)   
import glob
import pandas as pd
import numpy as np
import os
#지역 변수 선언
region = "충북"# "서울", 부산", "대구", "인천", "광주", "대전", "울산", "세종", "경기", "강원", "충북", "충남", "전북", "전남", "경북", "경남", "제주"
detail = "기상상태"

os.makedirs(f"C:/Users/HDW/Documents/project/data_raw/{region}",exist_ok= True)

acc1_files = glob.glob("C:/Users/HDW/Downloads/교통사고 지표 현황*xls")
tmp_raw = []

for name in acc1_files:
    tmp = pd.read_excel(name, header = 1,usecols=[0, 1, 2, 3, 4, 8])
    tmp_raw.append(tmp)

acc1_raw = pd.concat(tmp_raw)

acc1_raw.columns=["시도", "구", "연도", detail, "발생건수", "사망자수"]

acc1_raw = acc1_raw.dropna() #결측치 삭제

acc1_raw['연도'] = acc1_raw['연도'].astype(str).str.split('.').str[0] #연도 str화

# 피봇 테이블 직전 버전
acc1_raw.to_excel(f"../data_raw/{region}/{region}_{detail}_raw.xlsx", index=False) 

acc1 = acc1_raw.pivot_table( index = ["시도", "구", "연도"], columns = f"{detail}", values=['발생건수', '사망자수'], aggfunc = np.sum)
acc1

#평탄화 작업
new_columns = []

for num in range(int(len(acc1.columns)/2)):
    new_columns.append('발생_'+acc1.columns[num][1])

for num in range(int(len(acc1.columns)/2)):
    new_columns.append('사망_'+acc1.columns[num][1])

acc1.columns = new_columns

acc1.reset_index(inplace=True)

# 최종 버전
acc1.to_excel(f"../data_raw/{region}/{region}_{detail}.xlsx", index=False)

for name in acc1_files:
    os.remove(name)

# 기상청AWS 데이터 수집

In [ ]:
import warnings
warnings.filterwarnings(action = "ignore")
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import tqdm
from tqdm import tqdm_notebook

#브라우저 꺼짐 방지 
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
#불필요한 에러 메시지 없애기 
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

In [ ]:
class weather3:
    def __init__(self):
        self.first = True
    
    def weather_login(self):
        exception_l = [2, 85, 147, 191, 252, 259, 266, 271, 287, 317, 323, 332, 355, 434, 470, 511, 546]
        check_l = range(3,575)
        cc = [x for x in check_l if x not in exception_l]
        
        driver = webdriver.Chrome(options=chrome_options)
        driver.get("https://data.kma.go.kr/data/grnd/selectAwsRltmList.do?pgmNo=56")
        time.sleep(1)

        login_btn = driver.find_element(By.ID,"loginBtn")
        login_btn.click()
        time.sleep(0.5)

        input_id = driver.find_element(By.ID,"loginId")
        input_id.send_keys("dwstarwow@gmail.com")
        time.sleep(0.5)

        input_pw = driver.find_element(By.ID,"passwordNo")
        input_pw.send_keys("a*ehddn1212")
        input_pw.send_keys("\n")
        time.sleep(0.5)

        #close_btn = driver.find_element(By.ID,"loginPopCloseBtn")
        #close_btn.click()
        #time.sleep(0.5)

        temper_check = driver.find_element(By.ID,"ztree1_3_check")
        temper_check.click()
        time.sleep(0.5)

        wind_check = driver.find_element(By.ID,"ztree1_6_check")
        wind_check.click()
        time.sleep(0.5)

        rain_check = driver.find_element(By.ID,"ztree1_8_check")
        rain_check.click()
        time.sleep(0.5)
        for exp in exception_l:
            plus = driver.find_element(By.ID,f"ztree_{exp}_switch")
            plus.click()
            time.sleep(0.5)
        
        for i in tqdm_notebook(range(5)):
            year = str(i+2018)
            year_n = str(i+2019)
            start_d = driver.find_element(By.XPATH,"/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[1]/dl[2]/dd/div[1]/input")
            start_d.clear()
            start_d.send_keys(f"{year}0101")
            time.sleep(0.5)

            end_d = driver.find_element(By.XPATH,"/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[1]/dl[2]/dd/div[2]/input")
            end_d.clear()
            end_d.send_keys(f"{year_n}0101")
            time.sleep(0.5)
            end_d.send_keys("\n")
            time.sleep(0.5)
            
            for num in cc:
                loc_check = driver.find_element(By.ID,f"ztree_{str(num)}_check")
                driver.execute_script("arguments[0].scrollIntoView();", loc_check)
                loc_check.click()
                time.sleep(0.5)

                search = driver.find_element(By.XPATH,"/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[3]/button")
                search.click()
                time.sleep(20)

                excel = driver.find_element(By.XPATH,"/html/body/div[2]/div[1]/div/div[2]/div[3]/div[4]/div[1]/div/a[2]")
                excel.click()
                time.sleep(10)

                if self.first:
                    op1 = driver.find_element(By.ID,"reqstPurposeCd7")
                    op1.click()
                    time.sleep(0.5)

                    rq = driver.find_element(By.XPATH,"/html/body/div[5]/div/div/div/div/div[2]/div/a[2]")
                    rq.click()
                    self.first = False
                    time.sleep(10)
                for exp in exception_l:
                    plus = driver.find_element(By.ID,f"ztree_{exp}_switch")
                    plus.click()
                    time.sleep(0.5)
                
                loc_check = driver.find_element(By.ID,f"ztree_{str(num)}_check")
                driver.execute_script("arguments[0].scrollIntoView();", loc_check)
                loc_check.click()
                time.sleep(0.5)
            
        driver.close()

In [ ]:
a3 = weather3()

In [ ]:
a3.weather_login()